# <h0><center>Авиарейсы без потерь</center></h0>

![Image of Yaktocat](https://dailypix.ru/wp-content/uploads/2016/12/cdec8186dd8df7bf691ac877aa45f2a4.jpg)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from geopy import distance
from math import sin, cos, sqrt, radians, asin
from adjustText import adjust_text

In [2]:
df = pd.read_csv('query_result_8.csv')
df

,flight_id,flight_no,departure_airport,arrival_airport,actual_arrival,actual_departure,model,latitude,longitude,total_am_flight,count_pass
0,136119,PG0252,AAQ,SVO,2017-01-03T11:49:00Z,2017-01-03T10:08:00Z,Boeing 737-300,55.972599,37.414600,1653000.0,113
1,136120,PG0252,AAQ,SVO,2017-02-24T11:46:00Z,2017-02-24T10:07:00Z,Boeing 737-300,55.972599,37.414600,1605400.0,109
2,136122,PG0252,AAQ,SVO,2017-01-08T11:48:00Z,2017-01-08T10:08:00Z,Boeing 737-300,55.972599,37.414600,1431000.0,97
3,136130,PG0252,AAQ,SVO,2017-02-13T11:46:00Z,2017-02-13T10:07:00Z,Boeing 737-300,55.972599,37.414600,1556600.0,107
4,136131,PG0252,AAQ,SVO,2017-01-28T11:46:00Z,2017-01-28T10:07:00Z,Boeing 737-300,55.972599,37.414600,1812800.0,124
...,...,...,...,...,...,...,...,...,...,...,...
113,136936,PG0480,AAQ,EGO,2017-01-02T10:18:00Z,2017-01-02T09:28:00Z,Sukhoi Superjet-100,50.643799,36.590099,632400.0,82
114,136937,PG0480,AAQ,EGO,2017-01-03T10:19:00Z,2017-01-03T09:29:00Z,Sukhoi Superjet-100,50.643799,36.590099,626100.0,81
115,136951,PG0480,AAQ,EGO,2017-02-02T10:17:00Z,2017-02-02T09:28:00Z,Sukhoi Superjet-100,50.643799,36.590099,720600.0,90
116,136953,PG0480,AAQ,EGO,2017-02-27T10:17:00Z,2017-02-27T09:27:00Z,Sukhoi Superjet-100,50.643799,36.590099,765300.0,97


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   flight_id          118 non-null    int64  
 1   flight_no          118 non-null    object 
 2   departure_airport  118 non-null    object 
 3   arrival_airport    118 non-null    object 
 4   actual_arrival     118 non-null    object 
 5   actual_departure   118 non-null    object 
 6   model              118 non-null    object 
 7   latitude           118 non-null    float64
 8   longitude          118 non-null    float64
 9   total_am_flight    118 non-null    float64
 10  count_pass         118 non-null    int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 10.3+ KB


In [4]:
# Переводим переменные в формат datetime
df["actual_arrival"] = pd.to_datetime(df["actual_arrival"])
df["actual_departure"] = pd.to_datetime(df["actual_departure"])

# Считаем время проведенное в полете
df['flight_time'] = df['actual_arrival'] - df['actual_departure']

In [5]:
# Дополнительные данные:
# удельный расход топлива, кг/ч
fuel_consumption = {'Boeing 737-300': 2600, 'Sukhoi Superjet-100':1700}
df['fuel_consumption']= df['model'].map(fuel_consumption)

# средняя стоимость топлива в зимние месяцы, руб/т
fuel_cost = 42696

# Расчет цены авивтоплива за полет
def cost(col):
    duration_in_hour = float(col['flight_time']/ np.timedelta64(1, 'h')) # получаем часы из длительности в минутах
    cost_in_kg = fuel_cost/1000 # рассчитываем стоимость топлива в кг
    fuel_consump = col['fuel_consumption'] 
    return duration_in_hour*cost_in_kg*fuel_consump

In [6]:
# Вычисляем расстояние между Анапой и конечным пунктом в км

# approximate radius of earth in km 

R = 6373.0

def dist_calc (row):
    lon1 = 37.347301483154
    lat1 = 45.002101898193
    lat2 = row['latitude']
    lon2 = row['longitude']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))

    distance = R * c

    return distance


df['distance'] = df.apply (lambda row: dist_calc(row), axis=1)

In [7]:
#присвоим каждому полёту номер месяца
df['departure_month'] = df['actual_departure'].apply(lambda x: x.month)

In [8]:
# добавляем столбец с затратами на рейс деленное на кол-во пассажиров
df['flight_cost'] = df.apply(lambda x: cost(x), axis=1)

# считаем прибыльность рейсов
df['profit'] = df['total_am_flight'] - df['flight_cost']

In [13]:
#Общая прибыльность зимних рейсов.
profit_by_flight = pd.DataFrame(df.groupby(['departure_airport',
                                            'arrival_airport',
                                            'model',
                                            'flight_no']).profit.sum())
profit_by_flight

profit
departure_airport arrival_airport model               flight_no            
AAQ               EGO             Sukhoi Superjet-100 PG0480     38340977.4
                  SVO             Boeing 737-300      PG0252     86014557.6